# Running Drivers

Drivers can be run either from **bash** using the `uw` command-line interface (CLI), or from **Python** using the application programming interface (API) provided by the `uwtools.api` namespace. 

## Run chgres_cube for initial conditions

### With the CLI

To make the commands below copy-and-paste friendly, you'll need to export your working directory as an enviornment variable.

In [11]:
!export uw_training=$PWD

The `--help` or `-h` flag is available for all modes and actions.

In [12]:
!uw chgres_cube -h

usage: uw chgres_cube [-h] [--version] [--show-schema] TASK ...

Execute chgres_cube tasks

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --show-schema
      Show driver schema and exit

Positional arguments:
  TASK
    namelist_file
      The namelist file
    provisioned_rundir
      Run directory provisioned with all required content
    run
      A run
    runscript
      The runscript
    show_output
      Show the output to be created by this component
    validate
      Validate the UW driver config


Before running the driver, you can always validate your configuration file against our schema.

In [18]:
!cat $uw_training/configs/prepare_bcs.yaml

user:
  expt_dir: /scratch4/BMC/wrfruc/elcarpenter/uwtools_training/rundir
  uw_training: /scratch4/BMC/wrfruc/elcarpenter/uwtools_training
  rrfs_workflow: /scratch3/BMC/wrfruc/cholt/rrfs_work/rrfs-workflow
timevars:
  yyyymmddhh: '{{ cycle.strftime("%Y%m%d%H") }}'
  yyyymmdd: '{{ cycle.strftime("%Y%m%d") }}'
  hh: '{{ cycle.strftime("%H") }}'
  fff: '{{ "%03d" % (leadtime.total_seconds() / 3600) }}'
platform:
  fix: /scratch3/BMC/wrfruc/cholt/rrfs_work/FIX_RRFS
  fix_am: "{{ platform.fix }}/am"
  fix_lam: "{{ platform.fix }}/lam/RRFS_CONUS_13km_Lake_fracSV"
  input_data: /scratch3/BMC/wrfruc/cholt/rrfs_work/input_data
make_ics:
  # https://uwtools.readthedocs.io/en/main/sections/user_guide/yaml/components/chgres_cube.html
  chgres_cube:
    execution: &chgres_cube_execution
      batchargs:
        nodes: 1
        tasks_per_node: 20
        walltime: 00:30:00
      envcmds:
        - module use {{ user.rrfs_workflow }}/modulefiles
        - module load build_hera_intel
      executa

In [13]:
%%script bash -l
args=(
    -c $uw_training/configs/prepare_bcs.yaml
    --key-path make_ics
    --cycle 2025-08-27T12
    --leadtime 0
)
uw chgres_cube validate ${args[*]} 

[2025-09-23T14:39:54]     INFO Validating config against internal schema: chgres-cube
[2025-09-23T14:39:54]     INFO Schema validation succeeded for chgres_cube config
[2025-09-23T14:39:54]     INFO Validating config against internal schema: platform
[2025-09-23T14:39:54]     INFO Schema validation succeeded for platform config
[2025-09-23T14:39:54]     INFO 20250827 12:00:00 chgres_cube valid schema: Ready


The `provisioned_rundir` task will prepare the run directory with all required content.

In [14]:
%%script bash -l
args=(
    -c $uw_training/configs/prepare_bcs.yaml
    --key-path make_ics
    --cycle 2025-08-27T12
    --leadtime 0
    --batch
)
uw chgres_cube provisioned_rundir ${args[*]}

[2025-09-23T14:39:59]     INFO Validating config against internal schema: chgres-cube
[2025-09-23T14:39:59]     INFO Schema validation succeeded for chgres_cube config
[2025-09-23T14:39:59]     INFO Validating config against internal schema: platform
[2025-09-23T14:39:59]     INFO Schema validation succeeded for platform config
[2025-09-23T14:39:59]     INFO 20250827 12:00:00 chgres_cube namelist file fort.41: Executing
[2025-09-23T14:39:59]     INFO Schema validation succeeded for user-updated config
[2025-09-23T14:39:59]     INFO 20250827 12:00:00 chgres_cube namelist file fort.41: Ready
[2025-09-23T14:39:59]     INFO 20250827 12:00:00 chgres_cube runscript.chgres_cube: Executing
[2025-09-23T14:39:59]     INFO 20250827 12:00:00 chgres_cube runscript.chgres_cube: Ready
[2025-09-23T14:39:59]     INFO 20250827 12:00:00 chgres_cube provisioned run directory: Ready


And finally, to run the driver:

In [16]:
%%script bash -l
args=(
    -c $uw_training/configs/prepare_bcs.yaml
    --key-path make_ics
    --cycle 2025-08-27T12
    --leadtime 0
    --batch
)
uw chgres_cube run ${args[*]}

[2025-09-23T14:40:14]     INFO Validating config against internal schema: chgres-cube
[2025-09-23T14:40:14]     INFO Schema validation succeeded for chgres_cube config
[2025-09-23T14:40:14]     INFO Validating config against internal schema: platform
[2025-09-23T14:40:14]     INFO Schema validation succeeded for platform config
[2025-09-23T14:40:14]     INFO 20250827 12:00:00 chgres_cube namelist file fort.41: Executing
[2025-09-23T14:40:14]     INFO Schema validation succeeded for user-updated config
[2025-09-23T14:40:14]     INFO 20250827 12:00:00 chgres_cube namelist file fort.41: Ready
[2025-09-23T14:40:14]     INFO 20250827 12:00:00 chgres_cube runscript.chgres_cube: Executing
[2025-09-23T14:40:14]     INFO 20250827 12:00:00 chgres_cube runscript.chgres_cube: Ready
[2025-09-23T14:40:14]     INFO 20250827 12:00:00 chgres_cube provisioned run directory: Ready
[2025-09-23T14:40:14]     INFO 20250827 12:00:00 chgres_cube run via batch submission: Executing
[2025-09-23T14:40:14]     IN

### With the Python API

Docs: __[uwtools.api.chgres_cube](https://uwtools.readthedocs.io/en/main/sections/user_guide/api/chgres_cube.html)__

This is applicable to all drivers.

In [1]:
# Set up the environment using Python so it's available to each bash subshell below, along with all the Python cells.

import os
from datetime import datetime, timedelta
from pathlib import Path
uw_training = "/scratch4/BMC/wrfruc/elcarpenter/uwtools_training"
os.environ["uw_training"] = uw_training

cycle = datetime(2025, 8, 27, 12)

In [17]:
from uwtools.api.chgres_cube import ChgresCube
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

ics = ChgresCube(
    config=Path(uw_training, "configs", "prepare_bcs.yaml"),
    cycle=cycle,
    leadtime=timedelta(hours=0),
    key_path=["make_ics"],
)
ics.provisioned_rundir()

[2025-09-23T14:40:40]     INFO Validating config against internal schema: chgres-cube
[2025-09-23T14:40:40]     INFO Schema validation succeeded for chgres_cube config
[2025-09-23T14:40:40]     INFO Validating config against internal schema: platform
[2025-09-23T14:40:40]     INFO Schema validation succeeded for platform config
[2025-09-23T14:40:40]     INFO 20250827 12:00:00 chgres_cube namelist file fort.41: Executing
[2025-09-23T14:40:40]     INFO Schema validation succeeded for user-updated config
[2025-09-23T14:40:40]     INFO 20250827 12:00:00 chgres_cube namelist file fort.41: Ready
[2025-09-23T14:40:40]     INFO 20250827 12:00:00 chgres_cube runscript.chgres_cube: Executing
[2025-09-23T14:40:40]     INFO 20250827 12:00:00 chgres_cube runscript.chgres_cube: Ready
[2025-09-23T14:40:40]     INFO 20250827 12:00:00 chgres_cube provisioned run directory: Ready


20250827 12:00:00 chgres_cube provisioned run directory <140389620663008>

In [25]:
help(ics)

Help on ChgresCube in module uwtools.drivers.chgres_cube object:

class ChgresCube(uwtools.drivers.driver.DriverCycleLeadtimeBased)
 |  ChgresCube(cycle: 'datetime', leadtime: 'timedelta', config: 'dict | str | YAMLConfig | Path | None' = None, key_path: 'list[YAMLKey] | None' = None, batch: 'bool' = False, schema_file: 'Path | None' = None, controller: 'list[YAMLKey] | None' = None)
 |  
 |  A driver for chgres_cube.
 |  
 |  :param cycle: The cycle.
 |  :param leadtime: The leadtime.
 |  :param config: Path to config file (read stdin if missing or None).
 |  :param key_path: Keys of keys to driver config block.
 |  :param batch: Run component via the batch system?
 |  :param schema_file: Path to schema file to use to validate an external driver.
 |  :param controller: Key(s) leading to block in config controlling run-time values.
 |  
 |  Method resolution order:
 |      ChgresCube
 |      uwtools.drivers.driver.DriverCycleLeadtimeBased
 |      uwtools.drivers.driver.Driver
 |      u

In [26]:
ics.config

{'execution': {'batchargs': {'nodes': 1,
   'tasks_per_node': 20,
   'walltime': '00:30:00'},
  'envcmds': ['module use /scratch3/BMC/wrfruc/cholt/rrfs_work/rrfs-workflow/modulefiles',
   'module load build_hera_intel'],
  'executable': '/scratch3/BMC/wrfruc/cholt/rrfs_work/rrfs-workflow/exec/chgres_cube',
  'mpiargs': ['--export=ALL'],
  'mpicmd': 'srun'},
 'namelist': {'update_values': {'config': {'convert_atm': True,
    'convert_nst': False,
    'convert_sfc': True,
    'cycle_day': 27,
    'cycle_hour': 12,
    'cycle_mon': 8,
    'data_dir_input_grid': '/scratch3/BMC/wrfruc/cholt/rrfs_work/input_data/gfs',
    'external_model': 'GFS',
    'fix_dir_target_grid': '/scratch3/BMC/wrfruc/cholt/rrfs_work/FIX_RRFS/lam/RRFS_CONUS_13km_Lake_fracSV',
    'grib2_file_input_grid': 'GFS-20250827-12-f000.grib2',
    'halo_blend': 20,
    'halo_bndy': 4,
    'input_type': 'grib2',
    'lai_from_climo': True,
    'minmax_vgfrc_from_climo': True,
    'mosaic_file_target_grid': '/scratch3/BMC/wrfr

## Run chgres_cube for lateral boundary conditions

Run once for each lead time in a loop. Always point to the same section of the UW YAML.

In [ ]:
%%script bash -l
for i in $(seq 0 6) ; do 
  args=(
      -c $uw_training/configs/prepare_bcs.yaml
      --key-path make_lbcs
      --cycle 2025-08-27T12
      --leadtime $i
      --batch
  )
  uw chgres_cube run ${args[*]}
done

## Run a cold-start FV3 forecast

In [ ]:
%%script bash -l
args=(
  -c $uw_training/configs/fv3_config.yaml
  --key-path forecast_prod
  --cycle 2025-08-27T12
  --batch 
)
uw fv3 run ${args[*]}

## Run the analysis at the *NEXT* cycle

Here, the cycling frequency is 1 hour.

In [ ]:
%%script bash -l
# The notebook has a hard time displaying the output in a timely manner for the number of links required by GSI. Pipe to a file on disk.
args=(
    -c $uw_training/configs/gsi_config.yaml
    --key-path analysis
    --cycle 2025-08-27T13
    --batch
)
uw gsi run ${args[*]} > gsi.out 2>&1

In [ ]:
%%bash
cat gsi.out

## Run a cycled FV3 forecast from the updated analysis 

* This forecast was configured to run using the lateral boundary conditions from the previous cycle.
* It runs a script that removes the checksums from the analysis-update files between staging the files and submitting the job.

In [ ]:
%%bash
cat $uw_training/scripts/run_forecast.sh

In [ ]:
%%script bash -l
$uw_training/scripts/run_forecast.sh $uw_training/configs/fv3_config.yaml 2025-08-27T13